# Detect barcoded cells

In [ ]:
# imports and chamber selection
%load_ext autoreload
%autoreload 2
import iss_preprocess as iss
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml
from flexiznam.config import PARAMETERS
from pathlib import Path
from itertools import cycle
from matplotlib.animation import FuncAnimation
data_path = 'becalia_rabies_barseq/BRYC65.1d/chamber_13/'

processed_path = Path(PARAMETERS['data_root']['processed'])
metadata = iss.io.load_metadata(data_path)

ops = iss.config.DEFAULT_OPS.copy()
ops.update({ 
    'camera_order': metadata["camera_order"],
    'genes_rounds': metadata["genes_rounds"],
    'barcode_rounds': metadata["barcode_rounds"],
    'use_rois': [1, 2, 5, 6],
    'ref_tile': (1, 5, 8),
    'correction_tiles': [(1, 5, 8), (1, 5, 9), (1, 4, 8), (1, 4, 9), (2, 4, 9), (2, 3, 9), (2, 2, 9), (2, 2, 8)],
    'barcode_ref_tiles': [(1, 5, 8), (1, 5, 9), (1, 4, 8), (1, 4, 9), (2, 4, 9), (2, 3, 9), (2, 2, 9), (2, 2, 8)],
    'average_clip_value': 2000,
})

## Filter detected barcodes

We will load detected barcodes and filter them by dot product.

In [ ]:
roi = 5
all_spots = pd.read_pickle(processed_path / data_path / f'barcode_round_spots_{roi}.pkl')
print(f'{len(all_spots)} spots with {len(all_spots.bases.unique())} distincts barcodes.')

In [ ]:
threshold = 0.2
fig, ax = plt.subplots(1,1)
fig.set_size_inches((5,2))
ax.axvline(threshold, color='black')
ax.hist(all_spots.dot_product_score, 
            bins=np.arange(-0.5, 1,0.01), histtype='step')
ax.set_xlabel('Dot product score')
_=ax.set_ylabel('# spots')

In [ ]:
spots = all_spots[all_spots.dot_product_score > threshold]
print(f'{len(spots)} spots with {len(spots.bases.unique())} distincts barcodes.')

In [ ]:
# make a 1d kernel to convovle
import cv2
acq_data  = iss.io.load_single_acq_metdata(data_path, prefix='barcode_round_1_1')
pixel_size = acq_data['FrameKey-0-0-0']['PixelSizeUm']
kernel_size = int(10 / pixel_size)
kernel_size += (1 - kernel_size % 2) # kernel shape must be odd
kernel = cv2.getGaussianKernel(kernel_size, sigma=np.mean(kernel_size)/3)
# set the initial value so that single pixels after convolution have a peak of 1
kernel /= kernel.max()
kernel = kernel.astype("single")

fig = plt.figure(figsize=(5,1))
ax = fig.add_subplot(1,1,1)
ax.plot((np.arange(kernel_size)-kernel_size/2)*pixel_size,kernel)
_ = ax.set_xlabel('Distance (um)')

In [ ]:
pixel_size

In [ ]:
# make a spot "image"
print('Convolving')
blur = iss.segment.spots.make_spot_image(spots, kernel_size=kernel_size,
                                          dtype="single", output_shape=None)

In [ ]:
s = np.array([6000, 13000])
w = np.array([1800,1800])
b = np.vstack([s, s+w]).astype(int)
part2plot = (slice(*b[:,0]), slice(*b[:,1]))

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,2,1)
ax.imshow(blur[part2plot], vmax=10, vmin=0)
ax1 = fig.add_subplot(1,2,2)
snippet = np.array(blur[part2plot])
snippet[snippet < 2] = 0
ax1.imshow(snippet, vmin=0)

for x in [ax, ax1]:
    x.scatter(spots.x-s[1], spots.y-s[0],s=1, color='red', alpha=0.3)
    x.set_xlim(0, w[1])
    x.set_ylim(w[0],0)
    x.axis('off')
plt.tight_layout()


In [ ]:
print('Writing small snippet output')
from iss_preprocess.io.save import write_stack
stack = np.array(snippet, dtype=float)
stack *= ((2**16-1) / stack.max())
stack = stack[:,:, np.newaxis]
print(stack.shape)
write_stack(fname=processed_path/data_path/'barcode_spots_image.ome.tif', 
            stack=stack, bigtiff=False)

In [ ]:

cells = iss.segment.cells.cellpose_segmentation(
    snippet,
    channels=(0, 0),
    flow_threshold=0.5,
    min_pix=0,
    dilate_pix=0,
    rescale=None,
    model_type="CP",
    use_gpu=False,
    diameter=int(20/pixel_size)
)
cellpose_cells = np.array(cells)
plt.imshow(cells, cmap="Set2", interpolation="None")
print(f"Found {len(np.unique(cells))-1} cells")

# Opencv version

Now using classic opencv

In [ ]:
# binarise
binarise_threshold = 5
distance_threshold = 10
mask = 255 * (snippet > binarise_threshold).astype("uint8")
kernel = np.ones((5, 5), dtype='uint8') * 255
background = cv2.dilate(mask, kernel, iterations=10)
dst2nonzero = cv2.distanceTransform(mask, distanceType=cv2.DIST_L2, maskSize=5)
is_cell = 255 * (dst2nonzero > distance_threshold).astype("uint8")

In [ ]:
#plot various binary step
fig, axes = plt.subplots(2,2)
fig.set_size_inches(10, 10)
axes[0,0].imshow(mask)
axes[0,0].set_title('Binarised')
axes[0,1].imshow(background)
axes[0,1].set_title('Background is blue')
axes[1,0].imshow(dst2nonzero)
axes[1,0].set_title('Distance 2 non-zero')
axes[1,1].imshow(is_cell)
axes[1,1].set_title('Cells')

plt.tight_layout()

# Now watershed

We want to flood from each cell and extend around but not too far. We can do that by
setting a label for the background that is far from cells

In [ ]:
fig, axes = plt.subplots(2,2)
fig.set_size_inches(10, 10)

ret, markers = cv2.connectedComponents(np.array(is_cell, copy=True))
axes[0,0].imshow(markers, cmap='tab20', interpolation="None")
axes[0,0].set_title('Markers')

wshed_starting_pts = np.array(markers, copy=True)
wshed_starting_pts += 1 # make the background to 1
wshed_starting_pts[np.bitwise_xor(background, is_cell).astype(bool)] = 0  # and part to watershed to 0
axes[0,1].imshow(wshed_starting_pts, cmap='tab20', interpolation="None")
axes[0,1].set_title('Remove background')

stack = np.array(snippet, dtype=float)
stack *= ((2**8-1) / stack.max())
stack = cv2.cvtColor(stack.astype('uint8'), cv2.COLOR_GRAY2BGR)

water = cv2.watershed(stack, np.array(wshed_starting_pts, copy=True))
water[water == 1] = 0
axes[1,0].imshow(water, cmap="tab20", interpolation="None")
axes[1,0].set_title('Watershed')

snippet_with_borders = np.array(snippet)
snippet_with_borders[water==-1] = 10000
axes[1,1].imshow(wshed_starting_pts, cmap='tab20', interpolation="None")
axes[1,1].contour(water, colors="darkred")
axes[1,1].set_title('Cell contours')

for x in axes.flatten():
    x.axis('off')
plt.tight_layout()
print(f'Found {len(np.unique(water))-2} cells')
opencv_cells = np.array(water)


# Overlay to data

In [ ]:
# Get raw data for overlay
barcode, genes, angl, shift = iss.pipeline.stitch.stitch_and_register(
    data_path=data_path,
    reference_prefix="genes_round_1_1",
    target_prefix="barcode_round_1_1",
    roi=roi,
)


In [ ]:
# Get raw data for overlay
dapi, genes, angl, shift = iss.pipeline.stitch.stitch_and_register(
    data_path=data_path,
    reference_prefix="genes_round_1_1",
    target_prefix="dapi_1",
    roi=roi,
)


In [ ]:
borders = np.zeros(snippet.shape, dtype="uint8")
borders[water==-1] = 255
borders = cv2.dilate(borders, np.ones((5,5)))
img = iss.vis.to_rgb(np.dstack((dapi[part2plot], genes[part2plot], barcode[part2plot])), 
                     colors=[[0,0,1], [1,0,0], [0,1,0]], vmin=np.array([0,0, 70]), 
                     vmax=[500, 100, 200])

fig, axes = plt.subplots(1,2)

fig.set_size_inches(10, 6)
for x in axes:
    x.scatter(spots.x-s[1], spots.y-s[0],s=1, color='black', alpha=0.3)
    x.imshow(img)
    x.set_xlim(0, w[1])
    x.set_ylim(w[0],0)
    x.axis('off')
axes[0].set_title('CellPose')
axes[0].contour(cellpose_cells)

axes[1].set_title('OpenCV watershed')
axes[1].contour(opencv_cells)
plt.tight_layout()

In [ ]:
# basic imread
fname = 'barcode_round_1_1_MMStack_5-Pos000_000_fstack.tif'
full_fname = processed_path / data_path / "barcode_round_1_1" / fname
%timeit iss.io.load.load_stack(full_fname).astype('single')

In [ ]:
from tifffile import imread
%timeit np.moveaxis(imread(full_fname).astype('single'), 0, 2)

Optimising tile loading

In [ ]:
from skimage.morphology import binary_dilation

prefix='barcode_round'
tile_coors = (5, 0, 0)
nrounds = 1
suffix = "fstack"
filter_r = (2, 4)

## original version with just processing steps

This is for reference

In [ ]:
# origin version with just processing steps
def original_version():
    processed_path = Path(PARAMETERS["data_root"]["processed"])
    tforms_fname = f"tforms_corrected_{prefix}_{tile_coors[0]}_{tile_coors[1]}_{tile_coors[2]}.npz"
    tforms_path = processed_path / data_path / "reg" / tforms_fname
    tforms = np.load(tforms_path, allow_pickle=True)

    stack = iss.pipeline.load_sequencing_rounds(
        data_path, tile_coors, suffix=suffix, prefix=prefix, nrounds=nrounds
    )
    tforms = iss.pipeline.generate_channel_round_transforms(
        tforms["angles_within_channels"],
        tforms["shifts_within_channels"],
        tforms["scales_between_channels"],
        tforms["angles_between_channels"],
        tforms["shifts_between_channels"],
        stack.shape[:2],
    )
    stack = iss.pipeline.align_channels_and_rounds(stack, tforms)
    stack = iss.pipeline.apply_illumination_correction(data_path, stack, prefix)
    bad_pixels = np.any(np.isnan(stack), axis=(2, 3))
    stack[np.isnan(stack)] = 0
    stack = iss.pipeline.filter_stack(stack, r1=filter_r[0], r2=filter_r[1])
    mask = np.ones((filter_r[1] * 2 + 1, filter_r[1] * 2 + 1))
    bad_pixels = binary_dilation(bad_pixels, mask)

    correction_path = processed_path / data_path / f"correction_{prefix}.npz"
    norm_factors = np.load(correction_path, allow_pickle=True)["norm_factors"]
    stack = stack / norm_factors[np.newaxis, np.newaxis, :, :nrounds]
    return stack
%timeit original_version()


In [ ]:
# separate tform version with just processing steps
processed_path = Path(PARAMETERS["data_root"]["processed"])
tforms_fname = f"tforms_corrected_{prefix}_{tile_coors[0]}_{tile_coors[1]}_{tile_coors[2]}.npz"
tforms_path = processed_path / data_path / "reg" / tforms_fname
tforms = np.load(tforms_path, allow_pickle=True)
image_shape = (3300, 3296)
tforms = iss.pipeline.generate_channel_round_transforms(
    tforms["angles_within_channels"],
    tforms["shifts_within_channels"],
    tforms["scales_between_channels"],
    tforms["angles_between_channels"],
    tforms["shifts_between_channels"],
    image_shape,
)
def pregenerate_tforms():
    stack = iss.pipeline.load_sequencing_rounds(
        data_path, tile_coors, suffix=suffix, prefix=prefix, nrounds=nrounds
    )

    stack = iss.pipeline.align_channels_and_rounds(stack, tforms)
    stack = iss.pipeline.apply_illumination_correction(data_path, stack, prefix)
    bad_pixels = np.any(np.isnan(stack), axis=(2, 3))
    stack[np.isnan(stack)] = 0
    stack = iss.pipeline.filter_stack(stack, r1=filter_r[0], r2=filter_r[1])
    mask = np.ones((filter_r[1] * 2 + 1, filter_r[1] * 2 + 1))
    bad_pixels = binary_dilation(bad_pixels, mask)

    correction_path = processed_path / data_path / f"correction_{prefix}.npz"
    norm_factors = np.load(correction_path, allow_pickle=True)["norm_factors"]
    stack = stack / norm_factors[np.newaxis, np.newaxis, :, :nrounds]
%timeit pregenerate_tforms()


In [ ]:
iss.io.load.load_stack(full_fname).astype('single').shape

In [ ]:
np.moveaxis(imread(full_fname).astype('single'), 0, 2).shape

In [ ]:
roi = 5
stitched_stack_dapi, stitched_stack_genes, angle, shift = iss.pipeline.stitch_and_register(
    data_path, 'genes_round_1_1', 'DAPI_1', roi=roi, downsample=5
)


In [ ]:

stitched_stack_barcode, stitched_stack_genes, angle, shift = iss.pipeline.stitch_and_register(
    data_path, 'genes_round_1_1', 'barcode_round_1_1', roi=roi, downsample=5
)


In [ ]:
masks = np.load(processed_path / data_path / f"masks_{roi}.npy")
im = np.stack([
    stitched_stack_genes[3000:10000, 12000:20000], 
    stitched_stack_dapi[3000:10000, 12000:20000],
    masks[3000:10000, 12000:20000]>0],
    axis=2
)
shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=ops['ref_tile'], prefix='genes_round_1_1'
)
genes_spots = iss.pipeline.merge_roi_spots(
    data_path, shift_right, shift_down, tile_shape, iroi=roi, prefix="genes_round"
)


In [ ]:

barcode_spots = iss.pipeline.merge_roi_spots(
    data_path, shift_right, shift_down, tile_shape, iroi=roi, prefix="barcode_round"
)

plt.figure(figsize=(50,50))
plt.imshow(iss.vis.to_rgb(im, colors=[[1,0,0], [0,0,1], [0, 1, 0]], vmax=[400, 200, 1], vmin=np.array([30, 0, 0])))
plt.plot(barcode_spots["x"]-12000, barcode_spots["y"]-3000, '.r', alpha=1, markersize=10)
plt.plot(genes_spots["x"]-12000, genes_spots["y"]-3000, '.', color='purple', alpha=1, markersize=10)
plt.xlim([0, 4000])
plt.ylim([4000, 0])
plt.axis("off")